<a href="https://colab.research.google.com/github/jglobaton10/Databases-repository/blob/master/Analytics_engine_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit
!pip install streamlit-pandas-profiling
!pip install pyngrok

     |████████████████████████████████| 7.4MB 2.6MB/s 
     |████████████████████████████████| 102kB 11.3MB/s 
     |████████████████████████████████| 4.4MB 35.5MB/s 
     |████████████████████████████████| 163kB 44.7MB/s 
     |████████████████████████████████| 112kB 49.2MB/s 
     |████████████████████████████████| 133kB 46.0MB/s 
     |████████████████████████████████| 6.7MB 44.1MB/s 
     |████████████████████████████████| 122kB 44.1MB/s 
     |████████████████████████████████| 71kB 9.1MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=9a6c0fadb0157e261d8ebe775a597bae3400ac312248ab3d242038b3f1277f8c
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for blinker: filename=blinker-1.4-cp36-none-any.whl size=13450 sha256=02ee4f0cd02d93289ad1307d8e9381ebc9477a60939636e7024e54473a7c7f28
  Stored in directory: /root/.c

     |████████████████████████████████| 1.2MB 2.7MB/s 
  Created wheel for pyngrok: filename=pyngrok-4.2.2-cp36-none-any.whl size=18025 sha256=5f0931112c0954b42a683b7424f26559c5f826b84c03b65e8737f394ac2e061b
  Stored in directory: /root/.cache/pip/wheels/9c/d3/b5/502fbef4b2782a6142bc00c62ccde76ee04c9148fb43b1b2cd
Successfully built pyngrok


In [2]:
%%writefile app.py

import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import streamlit as st
from streamlit_pandas_profiling import st_profile_report
from pandas_profiling import ProfileReport


users = pd.read_csv('https://media.githubusercontent.com/media/jglobaton10/Databases-repository/master/Data%20pipeline/Users.csv', sep = ',')
activities = pd.read_csv('https://raw.githubusercontent.com/jglobaton10/Databases-repository/master/Data%20pipeline/Activities%20final%20final.csv')
user_does = pd.read_csv('https://raw.githubusercontent.com/jglobaton10/Databases-repository/master/Data%20pipeline/final_dos.csv')


data = pd.merge(users,user_does, left_on='id', right_on='id_user')
data = pd.merge(data, activities,  left_on='id_activity\t', right_on='id')
data.drop(['Date_y', 'Hour_y'],axis = 1, inplace = True)

st.write(
    """ 
    # Analytics questions 

    ## 1. Which activities are trending?

    """
)
x = data['id_activity\t'].value_counts().index
y = list(data['id_activity\t'].value_counts())
g = plt.figure(figsize=(5,5))
plt.scatter(x,y)
plt.xticks(x)
plt.grid(True)
plt.xticks(rotation=90)
plt.xlabel('Activity')
plt.ylabel('Amount')
st.pyplot(g)

st.write(
    """   
    ## 2. Which activities category is trending now?
    """
)

x = data['Category'].value_counts().index
y = list(data['Category'].value_counts())
g = plt.figure(figsize=(5,5))
plt.scatter(x,y, c = 'red')
plt.xticks(x)
plt.grid(True)
plt.xticks(rotation=90)
plt.xlabel('category')
plt.ylabel('Amount')

st.pyplot(g)

st.write(
    """   
    ## 3. Which are the most economic activities?
    """
)
x = list(activities['Name'])
y = list(activities['Cost'])
g = plt.figure(figsize=(5,5))
plt.bar(x,y)
plt.xticks(x)
plt.grid(True)
plt.xticks(rotation=90)
plt.xlabel('Name_activity')
plt.ylabel('Cost')
st.pyplot(g)

st.write(
    """   
    ## 4. Do we have common activities/events being attended to at certain times?
    """
)
x = data.groupby('Date_x').count()
x = x[x['Name'] > 5]
y = list(x['Name'])

g = plt.figure(figsize=(10,10))
plt.bar(x.index,y)
plt.xticks(x.index)
plt.grid(True)
plt.xticks(rotation=90)
plt.xlabel('Date')
plt.ylabel('Number of activities')
st.pyplot(g)

st.write(
    """   
    ## 5. Do we know which scenery/location do users prefer when attending certain events?
    """
)
x = data['Location'].value_counts()
y = list(x)
g = plt.figure(figsize=(10,10))
plt.bar(x.index,y)
plt.xticks(x.index)
plt.grid(True)
plt.xticks(rotation=90)
plt.xlabel('Location')
plt.ylabel('Number of activities')
st.pyplot(g)
st.write(
    """   
    ## 6. Does the user prefer doing some activities alone or with other people?
    """
)
x = data['Grupal'].value_counts()
y = list(x)
g = plt.figure(figsize=(10,10))
plt.bar(x.index,y)
plt.xticks(x.index)
plt.grid(True)
plt.xticks(rotation=90)
plt.xlabel('Modality')
plt.ylabel('Number of activities')
st.pyplot(g)
st.write(
    """   
    ## 7. Do we have common places being attended to at certain times?
    """
)
x = data.groupby(['Date_x', 'Location']).count()
x= x[x['Name'] > 1]['id_x']
st.write(x)

st.write(
    """   
    ## 8.	What is the price range the users are willing to pay for an activity?
    """
)
data['Cost'] = data['Cost'].str.replace('$', " ",regex = True)
data['Cost'] = data['Cost'].astype('float')
g = plt.figure(figsize=(10,10))
plt.hist(list(data['Cost']))
plt.xlabel('Price Range')
plt.ylabel('Frecuency')
st.pyplot(g)

st.write(
    """   
    ## 9.	Which type of events produce more profit?
    """
)
x = data.groupby('Category').sum()['Cost']
y = list(x)
g = plt.figure(figsize=(10,10))
plt.bar(x.index,y)
plt.xticks(x.index)
plt.grid(True)
plt.xticks(rotation=90)
plt.xlabel('Category')
plt.ylabel('Profit')
st.pyplot(g)

st.write(
    """   
    ## 10. Do certain discounts/coupons attract a considerable amount of users to a certain place?
    """
)


x = data.groupby('Location').sum()['Cost']
y = list(x)
g = plt.figure(figsize=(10,10))
plt.bar(x.index,y)
plt.xticks(x.index)
plt.grid(True)
plt.xticks(rotation=90)
plt.xlabel('Location')
plt.ylabel('Profit')
st.pyplot(g)

Writing app.py


In [3]:
!streamlit run app.py &>/dev/null&

In [4]:
from pyngrok import ngrok
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
public_url

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Support for "return_ngrok_tunnel" as "False" is deprecated and will be removed in 5.0.0, when this method will return a NgrokTunnel instead of a str
  This is separate from the ipykernel package so we can avoid doing imports until


'http://019f1b76f1eb.ngrok.io'